In [10]:
import nd2
import nd2.readers
import os
import matplotlib.pyplot as plt
import cv2
import shutil
import random
import numpy as np
import PySimpleGUI as sg
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import Canvas
import yaml
from ultralytics import YOLO
import torch
import torchvision

In [11]:
def get_template_files(file_path):
        files = os.listdir(f"{file_path}")
        files = [f for f in files if f.endswith(".png")]
        return files


def update_canvas(canvas, image_path):
    global img_width, img_height
    image = Image.open(image_path)
    img_width, img_height = image.size

    # キャンバスのサイズを画像のサイズに合わせる
    canvas.config(width=img_width, height=img_height)

    tk_image = ImageTk.PhotoImage(image)
    canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)
    canvas.image = tk_image  # Keep a reference to avoid garbage collection

# バウンディングボックスを描画するための関数
def draw_bbox(canvas, x1, y1, x2, y2):
    canvas.delete("bbox")  # Clear previous bbox
    canvas.create_rectangle(x1, y1, x2, y2, outline="red", tags="bbox")

# Canvasのマウスイベントをバインド
def on_canvas_button_press(event, window):
    global start_x, start_y
    start_x, start_y = event.x, event.y

def on_canvas_button_release(event, window):
    end_x, end_y = event.x, event.y
    
    # 画像のサイズを取得
    img_width, img_height = image.size
    
    # キャンバスのサイズを取得
    canvas_width, canvas_height = canvas.winfo_width(), canvas.winfo_height()
    
    # キャンバス上の座標を画像の実際のピクセル座標に変換
    start_x_img = start_x * img_width / canvas_width
    start_y_img = start_y * img_height / canvas_height
    end_x_img = end_x * img_width / canvas_width
    end_y_img = end_y * img_height / canvas_height
    
    # 相対座標に変換
    x_center = (start_x_img + end_x_img) / 2 / img_width
    y_center = (start_y_img + end_y_img) / 2 / img_height
    width = abs(end_x_img - start_x_img) / img_width
    height = abs(end_y_img - start_y_img) / img_height
    
    # YOLO形式の文字列を作成
    yolo_format = f"{x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    
    # 選択されたクラスに座標を追加
    if window["-class1_radio-"].get():  # Class1が選択されている場合
        current_values = window["-class1-"].get_list_values()
        window["-class1-"].update(current_values + [yolo_format])
    elif window["-class2_radio-"].get():  # Class2が選択されている場合
        current_values = window["-class2-"].get_list_values()
        window["-class2-"].update(current_values + [yolo_format])

    # 再度画像を更新してバウンディングボックスを描画
    img_path = os.path.join(current_dir, values["-name-"][0])
    img = Image.open(img_path)
    class1_coords = window["-class1-"].get_list_values()
    class2_coords = window["-class2-"].get_list_values()
    img_with_boxes = draw_bounding_boxes(img, class1_coords, class2_coords)
    update_canvas(canvas, img_with_boxes)

from PIL import Image, ImageDraw, ImageTk
import tkinter as tk

def draw_bounding_boxes(image, class1_coords, class2_coords):
    draw = ImageDraw.Draw(image)
    img_width, img_height = image.size
    
    # クラス1のバウンディングボックスを赤で描画
    for coord in class1_coords:
        x_center, y_center, width, height = map(float, coord.split())
        x_min = int((x_center - width / 2) * img_width)
        y_min = int((y_center - height / 2) * img_height)
        x_max = int((x_center + width / 2) * img_width)
        y_max = int((y_center + height / 2) * img_height)
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
    
    # クラス2のバウンディングボックスを青で描画
    for coord in class2_coords:
        x_center, y_center, width, height = map(float, coord.split())
        x_min = int((x_center - width / 2) * img_width)
        y_min = int((y_center - height / 2) * img_height)
        x_max = int((x_center + width / 2) * img_width)
        y_max = int((y_center + height / 2) * img_height)
        draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=2)
    
    return image

# txtファイルから既存の座標データを読み込む関数
def load_existing_annotations(txt_filepath):
    class1_coords = []
    class2_coords = []
    if os.path.exists(txt_filepath):
        with open(txt_filepath, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = parts[0]
                yolo_coord = " ".join(parts[1:])
                if class_id == "0":  # class1
                    class1_coords.append(yolo_coord)
                elif class_id == "1":  # class2
                    class2_coords.append(yolo_coord)
    return class1_coords, class2_coords

# Canvasに画像を更新する関数
def update_canvas(canvas, image):
    img_width, img_height = image.size
    canvas.config(width=img_width, height=img_height)
    img = ImageTk.PhotoImage(image)
    canvas.create_image(0, 0, anchor="nw", image=img)
    canvas.image = img

# txtファイルから既存の座標データを読み込む関数
def load_existing_annotations(txt_filepath):
    class1_coords = []
    class2_coords = []
    if os.path.exists(txt_filepath):
        with open(txt_filepath, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = parts[0]
                yolo_coord = " ".join(parts[1:])
                if class_id == "0":  # class1
                    class1_coords.append(yolo_coord)
                elif class_id == "1":  # class2
                    class2_coords.append(yolo_coord)
    return class1_coords, class2_coords

def adjust_brightness(image, factor):
    """Adjust image brightness."""
    return cv2.convertScaleAbs(image, alpha=factor, beta=0)

def add_salt_and_pepper_noise(image, amount=0.02):
    """Add salt and pepper noise to the image."""
    noisy_image = image.copy()
    total_pixels = image.size
    num_salt = int(total_pixels * amount * 0.5)
    num_pepper = int(total_pixels * amount * 0.5)

    salt_coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape]
    noisy_image[salt_coords[0], salt_coords[1]] = 255

    pepper_coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape]
    noisy_image[pepper_coords[0], pepper_coords[1]] = 0

    return noisy_image

def random_scale(image, labels, scale_range=(0.8, 1.2)):
    """Randomly scale an image and adjust labels."""
    h, w = image.shape[:2]
    scale = np.random.uniform(*scale_range)
    scaled_image = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

    new_h, new_w = scaled_image.shape[:2]
    pad_h = max(0, h - new_h)
    pad_w = max(0, w - new_w)
    scaled_image = cv2.copyMakeBorder(scaled_image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    new_labels = []
    for label in labels:
        cls, x_center, y_center, width, height = label
        x_center_scaled = x_center * scale
        y_center_scaled = y_center * scale
        width_scaled = width * scale
        height_scaled = height * scale
        if 0 <= x_center_scaled <= 1 and 0 <= y_center_scaled <= 1:
            new_labels.append([cls, x_center_scaled, y_center_scaled, width_scaled, height_scaled])

    return scaled_image, new_labels

def flip_image_and_labels(image, labels, flip_code):
    """Flip the image and adjust labels accordingly."""
    h, w = image.shape[:2]
    flipped_image = cv2.flip(image, flip_code)

    flipped_labels = []
    for label in labels:
        cls, x_center, y_center, width, height = label
        if flip_code == 1:
            x_center = 1 - x_center
        elif flip_code == 0:
            y_center = 1 - y_center
        elif flip_code == -1:
            x_center = 1 - x_center
            y_center = 1 - y_center
        flipped_labels.append([cls, x_center, y_center, width, height])

    return flipped_image, flipped_labels

def augment_image_and_labels(image_path, label_path, output_image_dir, output_label_dir, augmentations):
    """Perform augmentations on an image and its labels."""
    image = cv2.imread(image_path)
    with open(label_path, 'r') as f:
        labels = [[float(x) if i > 0 else int(x) for i, x in enumerate(line.strip().split())] for line in f]

    for aug in augmentations:
        if aug == 'flip':
            for flip_code in [1, 0, -1]:
                augmented_image, augmented_labels = flip_image_and_labels(image, labels, flip_code)
                save_augmented_data(augmented_image, augmented_labels, image_path, label_path, output_image_dir, output_label_dir, f'flip_{flip_code}')
        elif aug == 'brightness':
            for factor in [0.5, 1.5]:
                augmented_image = adjust_brightness(image, factor)
                save_augmented_data(augmented_image, labels, image_path, label_path, output_image_dir, output_label_dir, f'brightness_{factor}')
        elif aug == 'scale':
            augmented_image, augmented_labels = random_scale(image, labels)
            save_augmented_data(augmented_image, augmented_labels, image_path, label_path, output_image_dir, output_label_dir, 'scale')
        elif aug == 'noise':
            noisy_image = add_salt_and_pepper_noise(image, amount=0.02)
            save_augmented_data(noisy_image, labels, image_path, label_path, output_image_dir, output_label_dir, 'noise')

def save_augmented_data(image, labels, image_path, label_path, output_image_dir, output_label_dir, suffix):
    """Save augmented image and labels."""
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    image_output_path = os.path.join(output_image_dir, f'{base_name}_{suffix}.jpg')
    label_output_path = os.path.join(output_label_dir, f'{base_name}_{suffix}.txt')

    cv2.imwrite(image_output_path, image)

    with open(label_output_path, 'w') as f:
        for label in labels:
            f.write(' '.join(map(str, label)) + '\n')



In [12]:
import PySimpleGUI as sg
import os
from PIL import Image, ImageTk, ImageDraw

# クリックとリリースの座標を記録する変数
start_x = None
start_y = None

# 左側のカラムレイアウト
col_left = [
    [sg.Text("Image Name", justification="center", size=(20, 1))],
    [sg.Button("Train", key="-TrainImage-"), sg.Button("Val", key="-ValImage-")],
    [sg.Listbox(values=[], key="-name-", size=(20, 40), enable_events=True)]
]

# 中央のカラムレイアウト（画像表示）
col_middle = [
    [sg.Canvas(key="-canvas-",size=(2048,2048))]
]

# クラス表示用カラムレイアウト
col_class = [
    [sg.Text("Bounding Boxes (YOLO Format)", justification="center", size=(20, 1))],
    [sg.Radio("Class 1", "class_choice", key="-class1_radio-", default=True)],
    [sg.InputText(key='-INPUT1-', justification="center")],
    [sg.Listbox(values=[], key="-class1-", size=(20, 20), enable_events=True, expand_x=True)],
    [sg.Radio("Class 2", "class_choice", key="-class2_radio-")],
    [sg.InputText(key='-INPUT2-', justification="center")],
    [sg.Listbox(values=[], key="-class2-", size=(20, 20), enable_events=True, expand_x=True)]
]

# 右側のカラムレイアウト（操作ボタン）
col_right = [
    [sg.Text("Save Data", justification="center", size=(20, 1))],
    [sg.Button("Save", key="-save-")],
    [sg.Text("Data Augmentation", justification="center", size=(20, 1))],
    [sg.Button("Data Augmentation", key="-Augmentation-")],
    [sg.Text("Make.yaml", justification="center", size=(20, 1))],
    [sg.Button("yaml", key="-yaml-")],
    [sg.Text("Train", justification="center", size=(20, 1))],
    [sg.Button("Train", key="-train-")],
    [sg.Text("Go To Predict Mode", justification="center", size=(20, 1))],
    [sg.Button("Go To Predict", key="gotopredict")]
]

# ウィンドウのレイアウト
layout = [
    [sg.Column(col_left, element_justification='center'),
    sg.Column(col_middle, element_justification='center',scrollable=True,vertical_scroll_only=False,size=(512,512)),
    sg.Column(col_class, element_justification='center'),
    sg.Column(col_right, element_justification='center', vertical_alignment='top')]
]

# ウィンドウの作成
window = sg.Window("Train-Mode", layout=layout, finalize=True)

canvas_elem = window["-canvas-"]
canvas = canvas_elem.Widget


canvas.bind("<Button-1>", lambda event: on_canvas_button_press(event, window))
canvas.bind("<ButtonRelease-1>", lambda event: on_canvas_button_release(event, window))

png_path = "sample_img/sample_image.png"
# サンプル画像を表示
image = Image.open(png_path)
update_canvas(canvas, image)

# 選択されているディレクトリ（TrainまたはVal）を保持
current_dir = "Dataset/original_process/train/image/processed_image"

# イベントループ
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break

    elif event == "-TrainImage-":

        current_dir = "Dataset/original_process/train/image/processed_image"
        file_list = os.listdir(current_dir)
        window["-name-"].update(file_list)

    elif event == "-ValImage-":

        current_dir = "Dataset/val/images"
        file_list = os.listdir(current_dir)
        window["-name-"].update(file_list)

    elif event == "-name-":
        img_name = values["-name-"][0]
        img_path = os.path.join(current_dir, img_name)
        image = Image.open(img_path)
        
        # バウンディングボックスの座標を読み込む
        txt_filename = os.path.splitext(img_name)[0] + ".txt"
        train_txt_filepath = os.path.join("Dataset/original_process/train/label/processed_label", txt_filename)
        val_txt_filepath = os.path.join("Dataset/val/labels", txt_filename)
        
        if os.path.exists(train_txt_filepath) and "train"in img_path:
            txt_filepath = train_txt_filepath

        elif os.path.exists(val_txt_filepath) and "val" in img_path:
            txt_filepath = val_txt_filepath
        else:
            class1_coords, class2_coords = [], []
        
        if os.path.exists(txt_filepath):
            class1_coords, class2_coords = load_existing_annotations(txt_filepath)
        else:
            class1_coords, class2_coords = [], []
        
        # バウンディングボックスを描画して画像を更新
        img_with_boxes = draw_bounding_boxes(image, class1_coords, class2_coords)
        update_canvas(canvas, img_with_boxes)
        
        # リストボックスを更新
        window["-class1-"].update(class1_coords)
        window["-class2-"].update(class2_coords)
        
    elif event == "-save-":

        # 現在選択されている画像のファイル名からテキストファイル名を生成
        txt_filename = os.path.splitext(values["-name-"][0])[0] + ".txt"
        
        # 現在のディレクトリにおけるテキストファイルのフルパスを生成
        train_txt_filepath = os.path.join("Dataset/original_process/train/label/processed_label", txt_filename)
        val_txt_filepath = os.path.join("Dataset/val/labels", txt_filename)
        
        # テキストファイルが train または val ディレクトリに存在するか確認
        if os.path.exists(train_txt_filepath):
            txt_filepath = train_txt_filepath
        elif os.path.exists(val_txt_filepath):
            txt_filepath = val_txt_filepath
        else:
            sg.popup("対応するテキストファイルが見つかりません")
            continue
        
        # リストボックスから座標を取得
        class1_coords = window["-class1-"].get_list_values()
        class2_coords = window["-class2-"].get_list_values()

            
        # テキストファイルに座標を書き込む
        with open(txt_filepath, "w") as f:
            for coord in class1_coords:
                f.write(f"0 {coord}\n")  # Class 1 の座標はクラスID 0 で保存
            for coord in class2_coords:
                f.write(f"1 {coord}\n")  # Class 2 の座標はクラスID 1 で保存
        
        sg.popup("座標をテキストファイルに保存しました")
    
    elif event == "-Augmentation-":
        # 使用例
        image_dir = "Dataset/original_process/train/image/processed_image"
        label_dir = "Dataset/original_process/train/label/processed_label"
        output_image_dir = 'Dataset/train/images'
        output_label_dir = 'Dataset/train/labels'

        augmentations = ['flip', 'brightness', 'scale', 'noise']
        for file_name in os.listdir(image_dir):
            if file_name.endswith('.jpg') or file_name.endswith('.png'):
                image_path = os.path.join(image_dir, file_name)
                label_path = os.path.join(label_dir, os.path.splitext(file_name)[0] + '.txt')
                augment_image_and_labels(image_path, label_path, output_image_dir, output_label_dir, augmentations)
        sg.popup("augmentationが完了しました")

    if event == "-yaml-":
        # ユーザーの入力に基づいて新しいYAMLファイルを作成
        new_yaml_file_path = os.path.join(os.getcwd(), 'new_yolo.yaml')
        print(f"New YAML file path: {new_yaml_file_path}")
        print(f"Input 1: {values['-INPUT1-']}")
        print(f"Input 2: {values['-INPUT2-']}")

        try:
            # 新しいYAMLデータを作成
            new_data = {
                'train': 'Dataset/train/images',
                'val': 'Dataset/val/images',
                'nc': 2,  # クラス数を2に設定
                'names': {
                    0: values["-INPUT1-"],
                    1: values["-INPUT2-"]
                }
            }

            # 新しいYAMLファイルに書き込む
            with open(new_yaml_file_path, 'w') as file:
                yaml.dump(new_data, file, default_flow_style=False, sort_keys=False)

            sg.popup("新しいyamlファイルが正常に作成されました")

        except Exception as e:
            sg.popup(f"エラーが発生しました: {e}")

    if event == "-train-":
        # モデルのロード（例: YOLOv8n）
        model = YOLO('yolov8n.pt')
        answer = sg.popup_yes_no("学習済みモデルを選択しますか？")
        if answer == 'No':
            model = YOLO('yolov8n.pt')
        elif answer == 'Yes':
            model_path = sg.popup_get_file(".ptファイルを選択してください")
            if model_path and model_path.endswith('.pt'):
                sg.popup(f"{model_path}が正常に読み込まれました")
                model = YOLO(model_path)
            else:
                sg.popup("読み込みに失敗しました。yolov8n.ptを使って学習を開始します")
                model = YOLO('yolov8n.pt')

        # MPSデバイスの確認
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        sg.popup(f"使用するデバイス: {device}")

        # トレーニングの実行
        results = model.train(
            data=f'{os.getcwd()}/new_yolo.yaml',  # データセットの設定
            epochs=10,             # エポック数
            batch=16,              # バッチサイズ
            imgsz=416,             # 画像サイズ
            workers=16,            # ワーカー数
            device=device,         # デバイス設定（MPSまたはCPU使用）
            cache=False
        )

        # 学習完了後のモデル保存
        model_name = sg.popup_get_text("学習が完了しました。モデルの名前を記入してください")
        if model_name:
            model.save(f'Dataset/model/{model_name}.pt')
            sg.popup(f"モデルが保存されました: {model_name}.pt")
        else:
            sg.popup("モデル名が未入力のため、保存されませんでした。")


window.close()

Ultralytics 8.3.38 🚀 Python-3.12.6 torch-2.5.1 MPS (Apple M2)
engine/trainer: task=detect, mode=train, model=/Users/oikawakazutaka/Desktop/画像解析/Dataset/model/dapi_EEA1.pt, data=/Users/oikawakazutaka/Desktop/画像解析/new_yolo.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=16, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

train: Scanning /Users/oikawakazutaka/Desktop/画像解析/Dataset/train/labels... 952 images, 112 backgrounds, 0 corrupt: 100%|██████████| 952/952 [00:00<00:00, 3660.17it/s]

train: New cache created: /Users/oikawakazutaka/Desktop/画像解析/Dataset/train/labels.cache



val: Scanning /Users/oikawakazutaka/Desktop/画像解析/Dataset/val/labels... 13 images, 2 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<00:00, 778.59it/s]

val: New cache created: /Users/oikawakazutaka/Desktop/画像解析/Dataset/val/labels.cache


...

       3/10      2.84G       1.21      1.122      1.026         50        416: 100%|██████████| 60/60 [02:49<00:00,  2.83s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 2.650s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.92s/it]


                   all         13        145    0.00196    0.00446   0.000994   0.000298

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/10      2.99G      1.165      1.016      1.022        120        416:  98%|█████████▊| 59/60 [03:13<00:03,  3.72s/it]